In [167]:
import mlflow.sklearn
import pandas as pd
import plotly.graph_objects as go


import my_functions as mf

In [168]:
# Import data for prediction

data = pd.read_csv(f'./data/amazon_22_quarters.csv',
                   parse_dates=['EffectiveDate'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554079 entries, 0 to 554078
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   SKU                  554079 non-null  object        
 1   OfferTermCode        554079 non-null  object        
 2   RateCode             554079 non-null  object        
 3   PricePerUnit         554079 non-null  float64       
 4   instanceType         554079 non-null  object        
 5   instanceFamily       554079 non-null  object        
 6   LeaseContractLength  554079 non-null  int64         
 7   PurchaseOption       554079 non-null  object        
 8   OfferingClass        554079 non-null  object        
 9   Product Family       554079 non-null  object        
 10  Location             554079 non-null  object        
 11  Current Generation   554079 non-null  object        
 12  vCPU                 554079 non-null  int64         
 13  Memory        

In [169]:
data = data[['PricePerUnit', 'LeaseContractLength', 'PurchaseOption', 'vCPU',
             'Memory', 'operatingSystem', 'networkPerformance', 'StorageSize',
             'DiskType', 'Quarter', 'instanceType', 'instanceFamily', 'YearQuarter', 'year']]

In [170]:
count_instTypes = data['instanceType'].value_counts()

In [171]:
grouped_data = data.groupby(['instanceType', 'instanceFamily'])

df_grouped = grouped_data.mean().reset_index()

/tmp/ipykernel_873968/1787295621.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped = grouped_data.mean().reset_index()


In [172]:
# Take instanceType as input from the user

# instance_type = "g5.2xlarge"
# operating_system = 'Windows'
# purchase_option = "Partial Upfront"

instance_type = input("Enter the instance type: ")
operating_system = input("Enter the operating system: ")
purchase_option = input("Enter the purchase option: ")


# filter by instance type data
# filtered_data = data[(data['instanceType'] == instance_type)
#                      & (data['year'] > 2021)]

In [173]:
filtered_data = data[(data['instanceType'] == instance_type)
                     & (data['operatingSystem'] == operating_system)
                     & (data['PurchaseOption'] == purchase_option)
                     & (data['year'] > 2016)]

# Group the filtered data by YearQuarter and calculate the mean price
grouped_data = filtered_data.groupby('YearQuarter')['PricePerUnit', 'LeaseContractLength', 'PurchaseOption',
                                                    'vCPU', 'Memory', 'operatingSystem',
                                                    'networkPerformance', 'StorageSize',
                                                    'DiskType', 'Quarter', 'instanceType',
                                                    'instanceFamily', 'YearQuarter', 'year'].mean()

# Convert the grouped data to a DataFrame
grouped_data = grouped_data.reset_index()


grouped_data['year'] = grouped_data['year'].astype(int)

/tmp/ipykernel_873968/4022339667.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_data = filtered_data.groupby('YearQuarter')['PricePerUnit', 'LeaseContractLength', 'PurchaseOption',
/tmp/ipykernel_873968/4022339667.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_data = filtered_data.groupby('YearQuarter')['PricePerUnit', 'LeaseContractLength', 'PurchaseOption',


In [174]:
if 'PurchaseOption' not in grouped_data.columns:
    grouped_data.insert(3, 'PurchaseOption', purchase_option)

if 'operatingSystem' not in grouped_data.columns:
    grouped_data.insert(5, 'operatingSystem', operating_system)

DiskType = "SSD"
if 'DiskType' not in grouped_data.columns:
    grouped_data.insert(5, 'DiskType', DiskType)

In [175]:
# Create a new DataFrame with the same values as the previous row
new_rows = pd.concat([grouped_data.iloc[[-1]]] * 4, ignore_index=True)

# Set the YearQuarter values for the new rows
last_year_quarter = pd.Period(grouped_data['YearQuarter'].iloc[-1])
new_rows['YearQuarter'] = pd.period_range(
    start=last_year_quarter + 1, periods=4, freq='Q')

# Change the year in the new rows
new_rows['year'] = new_rows['year'] + 1


# Append the new rows to the original DataFrame
grouped_data = pd.concat([grouped_data, new_rows], ignore_index=True)

In [176]:
grouped_data.to_csv(
    f'./data/{instance_type}_{operating_system}_{purchase_option}.csv', index=False)